## Student Learning

This notebook will explore data on student learning. 
More specifically, it will explore data regarding:

- which course was seen and for how long
- which student saw which course
- Date on which the courses were watched

In [2]:

import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import import_ipynb
from Student_info import country_Student
from Student_info import student_info

ImportError: cannot import name 'country_Student' from 'Student_info' (Student_info.ipynb)

In [2]:
student_learning = pd.read_csv('dataSets/365_database/365_student_learning.csv')
student_learning

,student_id,course_id,minutes_watched,date_watched
0,258798,23,0.3,2022-01-01
1,258800,2,12.9,2022-04-01
2,258800,7,46.8,2022-01-03
3,258800,7,4.3,2022-01-04
4,258800,7,31.7,2022-01-07
...,...,...,...,...
64530,295509,23,0.8,2022-10-20
64531,295510,53,0.1,2022-10-20
64532,295511,7,7.8,2022-10-20
64533,295512,7,10.4,2022-10-20



 - The above dataframe <mark> student_learning </mark> (referred to as DataFrame going forward)contains several columns of interest. 

 - To further analyze the data in relation to data from this and other notebooks, we will have to split the initial data frame into three different dataframes based on categories.  

 - My initial idea is to create through conditional logic a dict with course IDs as the keys and categorical DFs as the values.  

 - However, I want research if there are any functions in pandas documentation.  
<br>

In [3]:
course_freq = pd.DataFrame(student_learning['course_id'].value_counts())
course_freq.head()

,course_id
7,15353
21,7046
14,6702
23,6032
16,4628


In [3]:
missing_country_df = student_info[student_info.isnull().any(axis=1)]
missing_country = missing_country_df['student_id'].tolist()
missing_country

[261725,
 262665,
 272144,
 278827,
 282654,
 282914,
 283704,
 283791,
 284550,
 284790,
 286959,
 290648,
 293967]

The above cell accesses the initial DataFrame from <mark> Student_info</mark> notebook.

And from the said dataframe, extracts a list of Student ID for which the countries are missing.

These student IDs must be removed from any DataFrame being worked on. Since country is the major category on which our Analysis will be built.

In [4]:
df = student_learning.copy()
df

,student_id,course_id,minutes_watched,date_watched
0,258798,23,0.3,2022-01-01
1,258800,2,12.9,2022-04-01
2,258800,7,46.8,2022-01-03
3,258800,7,4.3,2022-01-04
4,258800,7,31.7,2022-01-07
...,...,...,...,...
64530,295509,23,0.8,2022-10-20
64531,295510,53,0.1,2022-10-20
64532,295511,7,7.8,2022-10-20
64533,295512,7,10.4,2022-10-20


Created a copy of student_learning dataframe.

This is so the new dataframe can be used as an argument to automating the cleaning of the dataframe.

# Automated Cleaning

Following cells will contain codes to automate the cleaning of the student_learning dataframe. 

This involves several steps which are:
 - Obtaining the list of student IDs with no recorded countries.
   - This achieved through extracting a list of Student IDs from the imported dataframe student_info from Student_info.ipynb.
 - Duplicating the student_learning DataFrame to be passed as an arguement in the automated cleaning of the said dataframe.
   - This is because DataFrames are mutable objects, manipulation made through functions would be applied everywhere the object is iterated.Leading to circular reality of sorts?
   
 - ### Automated Cleaning
  - This would need to a function to read through the list <mark>missing_country</mark>. 
  - Delete the rows with student IDs from the dataframe, which are common to both the Dataframe and the missing_country list (Referred to as student list going forward).
 
 ##### Update 
 ###### I
 - There are other intermediate steps in this process.
 - Not every student ID in the missing_country is present in the student_learning DataFrame.
 - I surmise there is another step in which the index position of the student IDs common to missing_country and student_learning is extracted.
 - The deletion of rows from student_learning will be then based on these index position numbers.
 ###### II
 - Talked to a friend and he recommended using the <mark> isin() </mark> function and the ... converse operator?<mark> ~  </mark>
 - So my day and a half's time of research could have been replaced by one line of code.


In [5]:
print(missing_country)
# df
len(missing_country)

[261725, 262665, 272144, 278827, 282654, 282914, 283704, 283791, 284550, 284790, 286959, 290648, 293967]


13

### Step 1:

- Extract the index position of the rows with student IDs which are in the student list and contain it in a list.

The cell below will achieve that

In [6]:
# missing_country
# df
df_cleaned = df[~df['student_id'].isin(missing_country)]
student_learning_cleaned = df_cleaned.reset_index(drop=True)
student_learning_cleaned

,student_id,course_id,minutes_watched,date_watched
0,258798,23,0.3,2022-01-01
1,258800,2,12.9,2022-04-01
2,258800,7,46.8,2022-01-03
3,258800,7,4.3,2022-01-04
4,258800,7,31.7,2022-01-07
...,...,...,...,...
64486,295509,23,0.8,2022-10-20
64487,295510,53,0.1,2022-10-20
64488,295511,7,7.8,2022-10-20
64489,295512,7,10.4,2022-10-20


In [8]:
country = []
i = 0
n = len(student_learning_cleaned)-1

while (i<=n):
    x=student_learning_cleaned['student_id'].loc[i]
    y=country_Student[x]
    country.append(y)
    i=i+1

The above cell creates a list of countries corresponding to the student ID in the dataframe. It will be attached with 

In [17]:
df_country = pd.DataFrame(country,columns = ['country'])
df_country

,country
0,IN
1,CA
2,CA
3,CA
4,CA
...,...
64486,EG
64487,IN
64488,CM
64489,CA


In [18]:
frames = [student_learning_cleaned, df_country]

student_learning_by_country = pd.concat(frames,axis = 1)

student_learning_by_country

,student_id,course_id,minutes_watched,date_watched,country
0,258798,23,0.3,2022-01-01,IN
1,258800,2,12.9,2022-04-01,CA
2,258800,7,46.8,2022-01-03,CA
3,258800,7,4.3,2022-01-04,CA
4,258800,7,31.7,2022-01-07,CA
...,...,...,...,...,...
64486,295509,23,0.8,2022-10-20,EG
64487,295510,53,0.1,2022-10-20,IN
64488,295511,7,7.8,2022-10-20,CM
64489,295512,7,10.4,2022-10-20,CA
